Data Visualization and Exploration
CPSC 5530
CRN 21428
Hunter Harris: zgt795
Project:

Import Libraries

In [103]:
import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime

Load Data

In [104]:
fire_stations_df = pd.read_csv("Fire_Station_data.csv")
fire_incidents_df = pd.read_csv("Fire_Incidents_Detailed.csv")
budget_df = pd.read_csv("Chattanooga__Operating_Budget.csv")

/home/hunter/anaconda3/envs/data/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (2,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Set DataFrame Options

In [105]:
pd.set_option('display.max_columns', None)

Inspect Data Sets

In [106]:
# Print DataFrame information
print(fire_incidents_df.info())
print(fire_incidents_df['Specific Incident Code and Description'].value_counts().head(50))
print(fire_stations_df.info())
print(budget_df.info())

# Print DataFrames
print(fire_incidents_df.head(5))
print(fire_stations_df.head(5))
print(budget_df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703708 entries, 0 to 703707
Data columns (total 31 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   Incident Number                                  703708 non-null  object 
 1   Incident Date                                    703708 non-null  object 
 2   Shift Color                                      225885 non-null  object 
 3   Specific Incident Code and Description           697648 non-null  object 
 4   Generic Incident Code                            697648 non-null  float64
 5   Generic Incident Description                     697648 non-null  object 
 6   Incident Category                                693590 non-null  object 
 7   StreetNumber                                     703708 non-null  int64  
 8   StreetPrefix                                     125721 non-null  object 
 9   StreetName     

Clean budget DataFrame

In [107]:
# Filter out all departments except fire
budget_df = budget_df[budget_df['Department'] == 'Fire']

# Filter out all Programs except for Fire Stations
budget_df = budget_df[budget_df['Program'].str.contains("Fire Station")]
# budget_df = budget_df[budget_df['Fiscal Year'] == 2021]

# Extract numerical value from Program and convert to numeric
budget_df['Program'] = pd.to_numeric(budget_df['Program'].str.extract('(\d+)', expand=False))
budget_df.rename(columns={'Program': 'Station'}, inplace=True)

# Drop unneeded columns
budget_df.drop(budget_df.columns[[1, 5, 7, 11]], axis=1, inplace=True)
budget_df = budget_df.reset_index(drop=True)

# Calculate budget by station
station_budget = pd.DataFrame(budget_df.groupby(['Station'])['Approved Amount'].sum())
print(station_budget)

         Approved Amount
Station                 
1             19125788.0
3              5154481.0
4              5006923.0
5              8956324.0
6              4974232.0
7              9088177.0
8              5003168.0
9              5036425.0
10             5327438.0
11             1736018.0
12             4972456.0
13             8679699.0
14             4566592.0
15             4791725.0
16             4894706.0
17             4931118.0
19             7973684.0
20             4407538.0
21             5040169.0
22             4966765.0


Clean Fire Station DataFrame

In [108]:
# Drop unneeded columns
fire_stations_df.drop(fire_stations_df.columns[[1, 2, 3, 9, 10]], axis=1, inplace=True)

# Extract station number and rename column
fire_stations_df['NAME'] = pd.to_numeric(fire_stations_df['NAME'].str.extract('(\d+)', expand=False))
fire_stations_df.rename(columns={'NAME': 'Station'}, inplace=True)

# Add missing phone numbers
fire_stations_df['PHONE_NO'].replace(np.NAN, "423-643-5600", inplace=True)

# Separate GPS coordinates into separate columns
fire_stations_df['the_geom'] = fire_stations_df['the_geom'].str.replace('POINT \(', ' ')
fire_stations_df['the_geom'] = fire_stations_df['the_geom'].str.replace(')', ' ')
fire_stations_df['the_geom'] = fire_stations_df['the_geom'].str.strip()
new = fire_stations_df["the_geom"].str.split(" ", n = 1, expand = True)
fire_stations_df["Lng"]= new[0]
fire_stations_df["Lat"]= new[1]

# Drop original GPS column
fire_stations_df.drop(fire_stations_df.columns[[0]], axis=1, inplace=True)

# Add budget column
fire_stations_df['Budget'] = fire_stations_df['Station'].map(station_budget['Approved Amount'])
print(fire_stations_df)

                   ADDRESS         CITY STATE  ZIP_CODE      PHONE_NO  \
0       4510 Bonny Oaks Dr  Chattanooga    TN     37416  423-892-2487   
1             3250 8th Ave  Chattanooga    TN     37407  423-867-4622   
2         809 S. Willow St  Chattanooga    TN     37404  423-622-2314   
3         6144 Dayton Blvd  Chattanooga    TN     37415  423-842-2253   
4           1033 Lupton Dr  Chattanooga    TN     37415  423-875-8420   
5             5 Francis St  Chattanooga    TN     37419  423-643-5600   
6         6418 Hixson Pike       Hixson    TN     37343  423-643-5600   
7           2181 Curtis St  Chattanooga    TN     37406  423-622-1414   
8            910 Wisdom St  Chattanooga    TN     37406  423-622-2042   
9        3003 Cummings Hwy  Chattanooga    TN     37419  423-825-0436   
10       5004 Brunswick Ln       Hixson    TN     37343  423-870-0638   
11   7700 East Brainerd Rd  Chattanooga    TN     37421  423-892-5300   
12  2103 Hickory Valley Rd  Chattanooga    TN     3

/tmp/ipykernel_16349/4139552511.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  fire_stations_df['the_geom'] = fire_stations_df['the_geom'].str.replace('POINT \(', ' ')
/tmp/ipykernel_16349/4139552511.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  fire_stations_df['the_geom'] = fire_stations_df['the_geom'].str.replace(')', ' ')


Clean Fire Incident DataFrame

In [109]:
# Convert incident date to datatime object and create year column
fire_incidents_df['Incident Date'] = pd.to_datetime(fire_incidents_df['Incident Date'])
fire_incidents_df['Year'] = fire_incidents_df['Incident Date'].dt.year
fire_incidents_df['Month'] = fire_incidents_df['Incident Date'].dt.month
print(fire_incidents_df['Year'].value_counts())

# Keep only years with budget data available
fire_incidents_df = fire_incidents_df[fire_incidents_df['Year'].isin(list(budget_df['Fiscal Year']))]

# Extract station number and rename column
fire_incidents_df['Responding Unit District Station'] = pd.to_numeric(fire_incidents_df['Responding Unit District Station'].str.extract('(\d+)', expand=False))
fire_incidents_df.rename(columns={'Responding Unit District Station': 'Station'}, inplace=True)
fire_incidents_df.dropna(how='all', subset=['Station'], inplace=True)
fire_incidents_df['Station'] = fire_incidents_df['Station'].astype(int)

#  Keep only records of known firestations from fire_stations_df
fire_incidents_df = fire_incidents_df[fire_incidents_df['Station'].isin(list(fire_stations_df['Station']))]

# Add missing Data
col = ['Incident Category', 'Civilian Casualty', 'Firefighter Casualty', 'Property Loss', 'Property Value']
fire_incidents_df[col] = fire_incidents_df[col].replace(np.NAN, 0)

# Drop unneeded columns
fire_incidents_df.drop(fire_incidents_df.columns[[0, 2, 4, 7, 8, 9, 10, 11, 16, 18]], axis=1, inplace=True)

print(fire_incidents_df.isna().sum())

# Drop null values
print(fire_incidents_df.isna().sum())
# fire_incidents_df = fire_incidents_df.dropna()
fire_incidents_df = fire_incidents_df.reset_index(drop=True)

2021    47710
2019    43088
2018    42383
2017    41554
2016    40084
2015    39146
2020    38688
2011    36824
2012    36498
2010    36345
2014    36064
2013    35780
2009    32304
2008    28995
2007    27427
2005    27216
2006    27146
2004    21359
2003    20387
2002    16547
2001    15681
2022    12481
2026        1
Name: Year, dtype: int64
Incident Date                                          0
Specific Incident Code and Description               977
Generic Incident Description                         977
Incident Category                                      0
Civilian Casualty                                      0
Firefighter Casualty                                   0
Property Loss                                          0
Property Value                                         0
Responding Unit In or Out of Reporting District        0
Station                                                0
Turnout Time                                       13153
Travel Time               

Build List for interactive heatmap

In [110]:
lat_long_list = []

# Create list for
for x in fire_incidents_df['Year'].sort_values().unique():
    print(x)
    lat_long_list.append(fire_incidents_df.loc[fire_incidents_df['Year'] == x,['Latitude','Longitude']].groupby(['Latitude','Longitude']).sum().reset_index().values.tolist())


2015
2016
2017
2018
2019
2020
2021


Fire Station Visualizations

In [111]:
# Build Base Map
base_map = folium.Map(location=[35.043631, -85.309677], control_scale=True, zoom_start=11)

# Add labels for Fire Stations
for station in range(0, len(fire_stations_df)):
    html=f"""
        <h1> Station # {fire_stations_df.iloc[station]['Station']}</h1>
        <ul>
            <li>Address: {fire_stations_df.iloc[station]['ADDRESS']}</li>
            <li>City: {fire_stations_df.iloc[station]['CITY']}</li>
            <li>State: {fire_stations_df.iloc[station]['STATE']}</li>
            <li>Zip: {fire_stations_df.iloc[station]['ZIP_CODE']}</li>
            <li>Phone #: {fire_stations_df.iloc[station]['PHONE_NO']}</li>
            <li>Budget: $ {fire_stations_df.iloc[station]['Budget']}</li>
        </ul>
        """
    iframe = folium.IFrame(html=html, width=250, height=215)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(location=[fire_stations_df.iloc[station]['Lat'], fire_stations_df.iloc[station]['Lng']],
                  popup=popup,
                  icon=folium.DivIcon(html=f"""
                  <img src="https://i.imgur.com/zhm3d2b.png" width="40" height="40"/>
""")).add_to(base_map)

# Save Map for fire stations only
base_map.save(outfile= "Fire_Department_Only.html")

# Add Heatmap to basemap
HeatMapWithTime(lat_long_list, radius=10,index=[2015,2016,2017,2018,2019,2020,2021], auto_play=True, max_opacity=0.3).add_to(base_map)

# Save Map for heatmap and fire station
base_map.save(outfile= "Fire_Department_Heatmap_2015_2021.html")

# Show map
base_map